In [277]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import pylab as pl

%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Read the data

In [278]:
data = pd.read_csv("../Final_Data/final1213_reshaped_v2.csv")
data = data.iloc[:,1:]


data.Hour = data.Hour.astype('category', ordered = False)
data.Month = data.Month.astype('category', ordered = False)
data.WeekofYear = data.WeekofYear.astype('category', ordered = False)
data.DayofWeek = data.DayofWeek.astype('category', ordered = False)
data.Year = data.Year.astype('category', ordered = False)
data.Income = data.Income.astype('category', ordered = False)

In [279]:
data.head()

,Datetime,Dew_Point_F,Humidity,Temperature_F,Wind_Speed_MPH,Consumption,Income,Overall,Month,DayofWeek,Hour,WeekofYear,Year
0,2012-01-01 00:00:00,50.0,88,53.6,13.8,0.221192,1,0.231749,1,6,0,52,2012
1,2012-01-01 00:00:00,50.0,88,53.6,13.8,0.190114,2,0.231749,1,6,0,52,2012
2,2012-01-01 00:00:00,50.0,88,53.6,13.8,0.268072,3,0.231749,1,6,0,52,2012
3,2012-01-01 01:00:00,50.0,88,53.6,15.0,0.616456,1,0.550101,1,6,1,52,2012
4,2012-01-01 01:00:00,50.0,88,53.6,15.0,0.355447,2,0.550101,1,6,1,52,2012


In [280]:
data.Hour.cat.rename_categories(['Hour%d' % g for g in data.Hour.cat.categories], inplace=True)
data.Month.cat.rename_categories(['Month%d' % g for g in data.Month.cat.categories], inplace=True)
data.WeekofYear.cat.rename_categories(['WeekofYear%d' % g for g in data.WeekofYear.cat.categories], inplace=True)
data.DayofWeek.cat.rename_categories(['DayofWeek%d' % g for g in data.DayofWeek.cat.categories], inplace=True)
data.Year.cat.rename_categories(['Year%d' % g for g in data.Year.cat.categories], inplace=True)
data.Income.cat.rename_categories(['Income%d' % g for g in data.Income.cat.categories], inplace=True)


In [281]:
data.Hour = data.Hour.astype('str')
data.Month = data.Month.astype('str')
data.WeekofYear = data.WeekofYear.astype('str')
data.DayofWeek = data.DayofWeek.astype('str')
data.Year = data.Year.astype('str')
data.Income = data.Income.astype('str')
#data.stdorToU = data.stdorToU.astype('str')

In [282]:
data.head()

,Datetime,Dew_Point_F,Humidity,Temperature_F,Wind_Speed_MPH,Consumption,Income,Overall,Month,DayofWeek,Hour,WeekofYear,Year
0,2012-01-01 00:00:00,50.0,88,53.6,13.8,0.221192,Income1,0.231749,Month1,DayofWeek6,Hour0,WeekofYear52,Year2012
1,2012-01-01 00:00:00,50.0,88,53.6,13.8,0.190114,Income2,0.231749,Month1,DayofWeek6,Hour0,WeekofYear52,Year2012
2,2012-01-01 00:00:00,50.0,88,53.6,13.8,0.268072,Income3,0.231749,Month1,DayofWeek6,Hour0,WeekofYear52,Year2012
3,2012-01-01 01:00:00,50.0,88,53.6,15.0,0.616456,Income1,0.550101,Month1,DayofWeek6,Hour1,WeekofYear52,Year2012
4,2012-01-01 01:00:00,50.0,88,53.6,15.0,0.355447,Income2,0.550101,Month1,DayofWeek6,Hour1,WeekofYear52,Year2012


In [283]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52512 entries, 0 to 52511
Data columns (total 13 columns):
Datetime          52512 non-null object
Dew_Point_F       52512 non-null float64
Humidity          52512 non-null int64
Temperature_F     52512 non-null float64
Wind_Speed_MPH    51879 non-null float64
Consumption       52512 non-null float64
Income            52512 non-null object
Overall           52512 non-null float64
Month             52512 non-null object
DayofWeek         52512 non-null object
Hour              52512 non-null object
WeekofYear        52512 non-null object
Year              52512 non-null object
dtypes: float64(5), int64(1), object(7)
memory usage: 5.2+ MB


In [284]:
# splitting training and testing data
train = data[data.Year == 'Year2012']
valid = data[data.Year == 'Year2013']
train = train.dropna()
valid = valid.dropna()


In [285]:
valid.head()

,Datetime,Dew_Point_F,Humidity,Temperature_F,Wind_Speed_MPH,Consumption,Income,Overall,Month,DayofWeek,Hour,WeekofYear,Year
26283,2013-01-01 00:00:00,44.6,87,48.2,9.2,0.214885,Income1,0.250803,Month1,DayofWeek1,Hour0,WeekofYear1,Year2013
26284,2013-01-01 00:00:00,44.6,87,48.2,9.2,0.244850,Income2,0.250803,Month1,DayofWeek1,Hour0,WeekofYear1,Year2013
26285,2013-01-01 00:00:00,44.6,87,48.2,9.2,0.284734,Income3,0.250803,Month1,DayofWeek1,Hour0,WeekofYear1,Year2013
26286,2013-01-01 01:00:00,44.6,87,48.2,10.4,0.387206,Income1,0.442699,Month1,DayofWeek1,Hour1,WeekofYear1,Year2013
26287,2013-01-01 01:00:00,44.6,87,48.2,10.4,0.426540,Income2,0.442699,Month1,DayofWeek1,Hour1,WeekofYear1,Year2013


In [286]:
len(train)

25977

## Reference model (Cubic temperature, no wind or humidity)

In [287]:
# linear regression model
equation = "Consumption ~ np.power(Temperature_F, 3) + np.power(Temperature_F, 2) + Temperature_F + Income + DayofWeek + Hour + WeekofYear"

lm_ref = smf.ols(formula = equation, data = train).fit()
lm_ref.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Consumption   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     1658.
Date:                Mon, 12 Dec 2016   Prob (F-statistic):               0.00
Time:                        02:06:15   Log-Likelihood:                 35437.
No. Observations:               25977   AIC:                        -7.070e+04
Df Residuals:                   25891   BIC:                        -7.000e+04
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
Intercept                      0.2583      0.026      9.893      0.000         0.207     0.309
Income[T.Income2]              0.0367      0.001     38.930      0.000         0.035     0.038
Income[T.Income3]              0.1109      0.001    117.760      0.000         0.109     0.113
DayofWeek[T.DayofWeek1]       -0.0041      0.001     -2.860      0.004        -0.007    -0.001
DayofWeek[T.DayofWeek2]       -0.0044      0.001     -3.035      0.002        -0.007    -0.002
DayofWeek[T.DayofWeek3]       -0.0069      0.001     -4.795      0.000        -0.010    -0.004
DayofWeek[T.DayofWeek4]       -0.0066      0.001     -4.628      0.000        -0.009    -0.004
DayofWeek[T.DayofWeek5]        0.0039      0.001      2.682      0.007         0.001     0.007
DayofWeek[T.DayofWeek6]        0.0227      0.001     15.818      0.000         0.020     0.026
Hour[T.Hour1]                  0.0285      0.003     10.613      0.000         0.023     0.034
Hour[T.Hour10]                 0.0899      0.003     33.341      0.000         0.085     0.095
Hour[T.Hour11]                 0.0915      0.003     33.783      0.000         0.086     0.097
Hour[T.Hour12]                 0.1013      0.003     37.043      0.000         0.096     0.107
Hour[T.Hour13]                 0.1042      0.003     37.953      0.000         0.099     0.110
Hour[T.Hour14]                 0.0944      0.003     34.300      0.000         0.089     0.100
Hour[T.Hour15]                 0.0935      0.003     34.021      0.000         0.088     0.099
Hour[T.Hour16]                 0.1215      0.003     44.320      0.000         0.116     0.127
Hour[T.Hour17]                 0.1876      0.003     68.856      0.000         0.182     0.193
Hour[T.Hour18]                 0.2597      0.003     95.841      0.000         0.254     0.265
Hour[T.Hour19]                 0.2973      0.003    110.403      0.000         0.292     0.303
Hour[T.Hour2]                 -0.0447      0.003    -16.672      0.000        -0.050    -0.039
Hour[T.Hour20]                 0.2939      0.003    109.605      0.000         0.289     0.299
Hour[T.Hour21]                 0.2662      0.003     99.450      0.000         0.261     0.271
Hour[T.Hour22]                 0.2068      0.003     77.311      0.000         0.202     0.212
Hour[T.Hour23]                 0.1058      0.003     39.554      0.000         0.101     0.111
Hour[T.Hour3]                 -0.0890      0.003    -33.147      0.000        -0.094    -0.084
Hour[T.Hour4]                 -0.1096      0.003    -40.646      0.000        -0.115    -0.104
Hour[T.Hour5]                 -0.1056      0.003    -39.289      0.000        -0.111    -0.100
Hour[T.Hour6]                 -0.0587      0.003    -21.847      0.000        -0.064    -0.053
Hour[T.Hour7]                  0.0212      

In [288]:
len(lm_ref.params)

86

In [289]:
# predicting
prediction = lm_ref.predict(valid.iloc[:,2:12])

# MAPE
MAPE = np.mean(abs((prediction - valid.Consumption)/valid.Consumption))

print MAPE

0.136879299405


In [290]:
ref_R2adj = lm_ref.rsquared_adj
ref_MAPE = MAPE

print "REFERENCE"
print "Adj.R-squared = {0}    MAPE = {1}".format(ref_R2adj, ref_MAPE) 

REFERENCE
Adj.R-squared = 0.844305841234    MAPE = 0.136879299405


In [291]:
models = ['Reference (cubic temperature, no wind or humidity)']
equations = [equation]
AdjR2 = [ref_R2adj]
MAPE_ = [ref_MAPE]
LR = ['-']

## Tests

## 1 - Temperature^0

In [292]:
# linear regression model
equation = "Consumption ~ Income + DayofWeek + Hour + WeekofYear"

lm = smf.ols(formula = equation, data = train).fit()

# predicting
prediction = lm.predict(valid.iloc[:,2:12])

# MAPE
MAPE = np.mean(abs((prediction - valid.Consumption)/valid.Consumption))

models.append('Taking out Temperature')
equations.append(equation)
AdjR2.append(lm.rsquared_adj)
MAPE_.append(MAPE)
LR.append(lm_ref.compare_lr_test(lm))

print "Adj.R-squared = {0}    MAPE = {1}".format(lm.rsquared_adj, MAPE)
print "       Change: {0}    Change: {1}  LR:{2}".format(lm.rsquared_adj - ref_R2adj, MAPE - ref_MAPE, LR[-1])

Adj.R-squared = 0.837939201782    MAPE = 0.146843515558
       Change: -0.00636663945108    Change: 0.00996421615288  LR:(1044.116003744828, 4.839721152429259e-226, 3.0)


## 2 - Temperature^1

In [293]:
# linear regression model
equation = "Consumption ~ Income + Temperature_F + DayofWeek + Hour + WeekofYear"

lm = smf.ols(formula = equation, data = train).fit()

# predicting
prediction = lm.predict(valid.iloc[:,2:12])

# MAPE
MAPE = np.mean(abs((prediction - valid.Consumption)/valid.Consumption))

models.append('Linear Temperature')
equations.append(equation)
AdjR2.append(lm.rsquared_adj)
MAPE_.append(MAPE)
LR.append(lm_ref.compare_lr_test(lm))

print "Adj.R-squared = {0}    MAPE = {1}".format(lm.rsquared_adj, MAPE)
print "       Change: {0}    Change: {1}  LR:{2}".format(lm.rsquared_adj - ref_R2adj, MAPE - ref_MAPE, LR[-1])

Adj.R-squared = 0.843752897203    MAPE = 0.13727028037
       Change: -0.000552944030912    Change: 0.000390980965163  LR:(94.099814721645089, 3.685401931643193e-21, 2.0)


## 3 - Temperature^2

In [294]:
# linear regression model
equation = "Consumption ~ Income + np.power(Temperature_F,2) + Temperature_F + DayofWeek + Hour + WeekofYear"

lm = smf.ols(formula = equation, data = train).fit()

# predicting
prediction = lm.predict(valid.iloc[:,2:12])

# MAPE
MAPE = np.mean(abs((prediction - valid.Consumption)/valid.Consumption))

models.append('Quadratic Temperature')
equations.append(equation)
AdjR2.append(lm.rsquared_adj)
MAPE_.append(MAPE)
LR.append(lm_ref.compare_lr_test(lm))

print "Adj.R-squared = {0}    MAPE = {1}".format(lm.rsquared_adj, MAPE)
print "       Change: {0}    Change: {1}  LR:{2}".format(lm.rsquared_adj - ref_R2adj, MAPE - ref_MAPE, LR[-1])

Adj.R-squared = 0.843801673244    MAPE = 0.137467994139
       Change: -0.000504167989653    Change: 0.000588694733869  LR:(84.985981418314623, 3.0048797885743721e-20, 1.0)


## 4 - Temperature^3 + wind + humidity

In [295]:
# linear regression model
equation = "Consumption ~ Income + Wind_Speed_MPH + Humidity + np.power(Temperature_F,3) + np.power(Temperature_F,2) + Temperature_F + DayofWeek + Hour + WeekofYear"

lm = smf.ols(formula = equation, data = train).fit()

# predicting
prediction = lm.predict(valid.iloc[:,2:12])

# MAPE
MAPE = np.mean(abs((prediction - valid.Consumption)/valid.Consumption))

models.append('Cubic Temperature + wind + humidity')
equations.append(equation)
AdjR2.append(lm.rsquared_adj)
MAPE_.append(MAPE)
LR.append(lm.compare_lr_test(lm_ref))

print "Adj.R-squared = {0}    MAPE = {1}".format(lm.rsquared_adj, MAPE)
print "       Change: {0}    Change: {1}  LR:{2}".format(lm.rsquared_adj - ref_R2adj, MAPE - ref_MAPE, LR[-1])

Adj.R-squared = 0.846311942523    MAPE = 0.136598808823
       Change: 0.00200610128986    Change: -0.000280490581772  LR:(338.89242940195254, 2.5730248225802146e-74, 2.0)


In [296]:
lm.compare_lr_test(lm_ref)

(338.89242940195254, 2.5730248225802146e-74, 2.0)

## 5 - Taking out Income

In [297]:
# linear regression model
equation = "Consumption ~ np.power(Temperature_F,3) + np.power(Temperature_F,2) + Temperature_F + DayofWeek + Hour + WeekofYear"

lm = smf.ols(formula = equation, data = train).fit()

# predicting
prediction = lm.predict(valid.iloc[:,2:12])

# MAPE
MAPE = np.mean(abs((prediction - valid.Consumption)/valid.Consumption))

models.append('Taking out Income')
equations.append(equation)
AdjR2.append(lm.rsquared_adj)
MAPE_.append(MAPE)
LR.append(lm_ref.compare_lr_test(lm))

print "Adj.R-squared = {0}    MAPE = {1}".format(lm.rsquared_adj, MAPE)
print "       Change: {0}    Change: {1}  LR:{2}".format(lm.rsquared_adj - ref_R2adj, MAPE - ref_MAPE, LR[-1])

Adj.R-squared = 0.757742118858    MAPE = 0.166464276911
       Change: -0.0865637223758    Change: 0.0295849775064  LR:(11486.677803586004, 0.0, 2.0)


In [298]:
results = pd.concat([pd.Series(models), pd.Series(equations), pd.Series(AdjR2),
                     pd.Series(MAPE_), pd.Series(LR)], axis=1)
results.rename(columns={0: 'model' , 1: 'equation', 2:'Adj-R2', 3: 'MAPE', 4: 'LR_test'}, inplace=True)

In [299]:
results

,model,equation,Adj-R2,MAPE,LR_test
0,"Reference (cubic temperature, no wind or humid...","Consumption ~ np.power(Temperature_F, 3) + np....",0.844306,0.136879,-
1,Taking out Temperature,Consumption ~ Income + DayofWeek + Hour + Week...,0.837939,0.146844,"(1044.11600374, 4.83972115243e-226, 3.0)"
2,Linear Temperature,Consumption ~ Income + Temperature_F + DayofWe...,0.843753,0.137270,"(94.0998147216, 3.68540193164e-21, 2.0)"
3,Quadratic Temperature,"Consumption ~ Income + np.power(Temperature_F,...",0.843802,0.137468,"(84.9859814183, 3.00487978857e-20, 1.0)"
4,Cubic Temperature + wind + humidity,Consumption ~ Income + Wind_Speed_MPH + Humidi...,0.846312,0.136599,"(338.892429402, 2.57302482258e-74, 2.0)"
5,Taking out Income,"Consumption ~ np.power(Temperature_F,3) + np.p...",0.757742,0.166464,"(11486.6778036, 0.0, 2.0)"


In [300]:
results.to_csv("../Final_Data/table_for_feature_selection.csv")